# Analiza skupień

Ładujemy niezbędne pakiety

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import numpy as np
import pandas as pd
np.set_printoptions(suppress=True)

Wczytujemy dane

In [ ]:
klienci_banku = pd.read_csv('banking_data.csv')

Wyświetlamy pierwsze wiersze

In [ ]:
klienci_banku.head()

Podsumowanie statystyczne ramki danych

In [ ]:
klienci_banku.describe()

Stwórz punktowy wykres obserwacji 3D

In [ ]:
fig = plt.figure(figsize=(10,8))
ax = Axes3D(fig)
ax.scatter(klienci_banku['zarobki'], klienci_banku['oszczednosci'], klienci_banku['wiek'])
plt.show()

Histogram wieku klientów

In [ ]:
klienci_banku['wiek'].hist(bins=12)
plt.title('Rozkład wieku klientów')
plt.grid(None)
plt.show()

Histogram udziału oszczędności w dochodach

In [ ]:
klienci_banku['oszczednosci'].hist(bins=20)
plt.title('Rozkład udziału oszczędności w zarobkach klientów')
plt.grid(None)
plt.show()

Rozdkład zarobków klientów

In [ ]:
klienci_banku['zarobki'].hist(bins=15)
plt.title('Rozkład zarobków klientów')
plt.grid(None)
plt.show()

Zdefiniuj funkcję przyjmującą dwa wektory i zwracającą odległośc euklidesową między nimi. Oblicz odległość dla przykładowych wektorów x = [10,20], y=[10,22]

In [ ]:
def odleglosc_euklidesowa(obserwacja1, obserwacja2):
    return np.sum((obserwacja1 - obserwacja2) ** 2)

odleglosc_euklidesowa(np.array([10,20]), np.array([10,22]))

Oblicz odległość euklidesową między pierwszym a drugim klientem banku

In [ ]:
print(klienci_banku.iloc[0,:])
print(klienci_banku.iloc[1,:])
odleglosc_euklidesowa(klienci_banku.iloc[0], klienci_banku.iloc[1])

Dokonaj standaryzacji danych (od każdej zmiennej odejmij średnią, i podziel przez jej odchylenie standardowe)

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
klienci_banku[klienci_banku.columns] = scaler.fit_transform(klienci_banku)


# ręcznie wyglądałoby to tak:
# ((klienci_banku - klienci_banku.mean()) / klienci_banku.std(ddof=0)).head()

In [ ]:
klienci_banku.head()

Dokonaj klastrowania metodą k-średnich. Przyjmij liczbę skupień jako 3

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=3)
kmeans.fit(klienci_banku)
print(kmeans)

Do którego skupienia przyporządkowano poszczególne obserwacje?

In [ ]:
kmeans.labels_

Jakie są centra skupień? jak byś je scharakteryzował?

In [ ]:
kmeans.cluster_centers_

In [ ]:
scaler.inverse_transform(kmeans.cluster_centers_)

Przedstaw na wykresie 3D dane, a kolorem oznacz numer skupiska

In [ ]:
fig = plt.figure(figsize=(10,8))
ax = Axes3D(fig)

ax.scatter(
    klienci_banku['zarobki'],
    klienci_banku['oszczednosci'],
    klienci_banku['wiek'],
    c=kmeans.labels_
)
plt.title('Wykres punktowy klientów banku')
plt.show()

Przedstaw miary jakości klastrowania dla tego modelu

In [ ]:
from sklearn import metrics

In [ ]:
kmeans.inertia_

In [ ]:
metrics.silhouette_score(klienci_banku, kmeans.labels_, metric='euclidean')

In [ ]:
metrics.calinski_harabaz_score(klienci_banku, kmeans.labels_)

Stwórz pętlę, która będzie tworzyłą modele k-średnich dla liczby skupień z zakresu (2,12), i zapisz po kolei miary jakości dopasowania do kolejnych list

In [ ]:
inertia_scores = []
silhouette_scores = []
calinski_harabaz_scores = []

for i in range(2,12):
    kmeans = KMeans(n_clusters=i)
    kmeans.fit(klienci_banku)
    inertia_scores.append(kmeans.inertia_)
    silhouette_scores.append(metrics.silhouette_score(klienci_banku, kmeans.labels_, metric='euclidean'))
    calinski_harabaz_scores.append(metrics.calinski_harabaz_score(klienci_banku, kmeans.labels_))
    

In [ ]:
plt.scatter(x=np.arange(2,12), y=inertia_scores)

In [ ]:
plt.scatter(x=np.arange(2,12), y=silhouette_scores)

In [ ]:
plt.scatter(x=np.arange(2,12), y=calinski_harabaz_scores)

Samodzielnie. Dokonaj grupowania metodą k-średnich za pomocą odpowiednio wybranej liczby skupień. Jak scharakteryzujesz tak powstałe grupy klientów banku?

In [ ]:
# Twój kod

#### Metoda aglomeracyjna

Przeprowadź proces grupowania za pomocą metody aglomeracyjnej. Wykorzystaj metrykę euklidesową, a jako wiązanie - metodę Warda. Wykreśl dendrogram, i wizualnie oceń na ile klastrów powinno podzielić się daną zbiorowość

In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster

In [ ]:
linkage_matrix = linkage(klienci_banku, 'ward')
plt.figure(figsize=(14,6))
dendrogram(linkage_matrix, color_threshold=0)
plt.show()

Przyporządkuj każdą obserwację do odpowiedniego klastra (przyjmij 3 klastry)

In [ ]:
numer_klastra = fcluster(linkage_matrix, t=3, criterion='maxclust')
print(numer_klastra)

In [ ]:
srednie = klienci_banku.groupby(numer_klastra).mean()
srednie

In [ ]:
scaler.inverse_transform(srednie)

In [ ]:
fig = plt.figure(figsize=(10,8))
ax = Axes3D(fig)

ax.scatter(
    klienci_banku['zarobki'],
    klienci_banku['oszczednosci'],
    klienci_banku['wiek'],
    c=numer_klastra
)
plt.title('Wykres punktowy klientów banku')
plt.show()

Samodzielnie - za pomocą analizy skupień scharakteryzuj dane dotyczące sytuacji mieszkaniowej w kalifornii (Zbiór danych California Housing dataset)

In [ ]:
california_housing = pd.read_csv('california_housing.csv')